<a href="https://colab.research.google.com/github/ridhixmakharia/Face-Recognition/blob/main/Face_Recognition_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
drive_path = "/content/drive/MyDrive"

# Imports

In [2]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

# Seeding

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [4]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

# Hyperparameters

In [5]:
batch_size = 4
lr = 1e-4
epochs = 50
height = 768
width = 512

# Path

In [6]:
dataset_path = os.path.join(drive_path, "Dataset", "aug")
files_dir = os.path.join("Colab Notebooks","files", "aug")
model_file = os.path.join(files_dir, "unet-aug.h5")
log_file = os.path.join(files_dir, "log-aug.csv")

# Creating Folder

In [7]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [8]:
create_dir(files_dir)

# Building UNET

## Conv Block

In [9]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    #padding is the same to maintain the same shape for the putput feature map as the input
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    #padding is the same to maintain the same shape for the putput feature map as the input
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

## Encoder

In [10]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2,2))(x)
    return x, p

## Decoder

In [11]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

## UNET

In [12]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    """ ENCODER """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """ BRIDGE """
    b1 = conv_block(p4, 1024)

    """ DECODER """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding = "same", activation = "sigmoid")(d4)
    model = Model(inputs, outputs, name="UNET")
    return model

# Dataset Pipeline

## Loading training and validation dataset

In [13]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train","images", "*")))
    train_y = sorted(glob(os.path.join(path, "train","masks", "*")))

    valid_x = sorted(glob(os.path.join(path, "valid","images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid","masks", "*")))

    return (train_x, train_y), (valid_x, valid_y)

## Reading Images

In [14]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    return x

## Reading Mask

In [15]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

## tf.data.pipeline

In [16]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x,y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

In [17]:
def tf_dataset(x, y, batch = 8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse, num_parallel_calls = tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Training

In [18]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 300 - 300
Valid: 19 - 19


In [19]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [20]:
print(train_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 768, 512, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 768, 512, 1), dtype=tf.float64, name=None))>


In [21]:
#for x, y in train_dataset:
    #print(x, y)

In [22]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [23]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 768, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 768, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 768, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 768, 512, 64)         0         ['batch_normalization[0][0]

In [24]:
opt = tf.keras.optimizers.legacy.Adam(lr)
model.compile(loss = "binary_crossentropy",optimizer = opt, metrics = ["acc"] )

In [25]:
callbacks = [
    ModelCheckpoint(model_file, verbose = 1, save_best_only = True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss',patience=20, restore_best_weights=False)
]

In [26]:
model.fit(
    train_dataset,
    validation_data = valid_dataset,
    epochs = epochs,
    callbacks=callbacks
)

Epoch 1/50
75/75 [==============================] - ETA: 0s - loss: 0.5796 - acc: 0.7310
Epoch 1: val_loss improved from inf to 0.61234, saving model to Colab Notebooks/files/aug/unet-aug.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


75/75 [==============================] - 203s 2s/step - loss: 0.5796 - acc: 0.7310 - val_loss: 0.6123 - val_acc: 0.8341 - lr: 1.0000e-04
Epoch 2/50
75/75 [==============================] - ETA: 0s - loss: 0.3954 - acc: 0.8514
Epoch 2: val_loss improved from 0.61234 to 0.49179, saving model to Colab Notebooks/files/aug/unet-aug.h5
75/75 [==============================] - 106s 1s/step - loss: 0.3954 - acc: 0.8514 - val_loss: 0.4918 - val_acc: 0.8350 - lr: 1.0000e-04
Epoch 3/50
75/75 [==============================] - ETA: 0s - loss: 0.3525 - acc: 0.8631
Epoch 3: val_loss improved from 0.49179 to 0.43429, saving model to Colab Notebooks/files/aug/unet-aug.h5
75/75 [==============================] - 106s 1s/step - loss: 0.3525 - acc: 0.8631 - val_loss: 0.4343 - val_acc: 0.8351 - lr: 1.0000e-04
Epoch 4/50
75/75 [==============================] - ETA: 0s - loss: 0.3406 - acc: 0.8668
Epoch 4: val_loss did not improve from 0.43429
75/75 [==============================] - 104s 1s/step - loss: 0

Import


In [27]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

Seeding

In [28]:
os.environ[ "PYTHONHASHSEED" ] = str(42)
np. random. seed (42)
tf. random.set_seed (42)

Hyperparameters

In [29]:
height = 769
width= 512


Path


In [30]:
dataset_path = "/content/drive/MyDrive/Dataset/aug/test"
save_path = "Colab Notebooks/prediction/aug"
model_path = "Colab Notebooks/files/aug/unet-aug.h5"

In [31]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)


In [32]:
create_dir(save_path)

In [33]:
model = tf.keras.models.load_model(model_path)

In [34]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 768, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 768, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 768, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 768, 512, 64)         0         ['batch_normalization[0][0]

In [35]:
test_x = sorted(glob(os.path.join(dataset_path,"images", "*")))
print (f"Test Images: {len(test_x)}")

Test Images: 19


In [36]:
test_x[0]

'/content/drive/MyDrive/Dataset/aug/test/images/118_0.png'

In [37]:
time_taken = []
for x in tqdm(test_x):
  name =x.split("/")[-1]

  x= cv2.imread(x, cv2.IMREAD_COLOR)
  x = x / 255.0
  x = np. expand_dims(x, axis=0)
  start_time = time.time()
  p = model. predict(x)[0]
  total_time = time. time() - start_time
  time_taken.append(total_time)
  p = p > 0.5
  p = p * 255
  cv2.imwrite(os.path.join(save_path, name), p)

  0%|          | 0/19 [00:00<?, ?it/s]

1/1 [==============================] - 7s 7s/step


  5%|▌         | 1/19 [00:11<03:22, 11.24s/it]

1/1 [==============================] - 0s 31ms/step


 11%|█         | 2/19 [00:11<01:25,  5.02s/it]

...

 89%|████████▉ | 17/19 [00:21<00:01,  1.84it/s]

1/1 [==============================] - 0s 23ms/step


 95%|█████████▍| 18/19 [00:22<00:00,  2.01it/s]

1/1 [==============================] - 0s 26ms/step


100%|██████████| 19/19 [00:23<00:00,  1.22s/it]


In [38]:
mean_time = np.mean (time_taken)
mean_fps = 1/mean_time

In [39]:
mean_time = np.mean (time_taken)
mean_fps = 1/mean_time
print(f"Mean Time: {mean_time: 1.5f} - Mean FPS: {mean_fps:2.5f}")

Mean Time:  0.75972 - Mean FPS: 1.31627


In [40]:
from glob import glob
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

In [41]:
pred_mask = sorted(glob(os.path.join("Colab Notebooks","prediction", "aug", "*")))
#true_mask = sorted(glob(os.path.join("/content/drive/MyDrive/", "Dataset","test", "masks", "*")))
true_mask = sorted(glob("/content/drive/MyDrive/Dataset/aug/test/masks/*"))

In [42]:
print(true_mask)

['/content/drive/MyDrive/Dataset/aug/test/masks/118_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/124_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/147_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/154_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/161_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/162_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/171_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/184_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/188_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/19_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/219_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/225_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/261_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/262_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/301_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/303_0.png', '/content/drive/MyDrive/Dataset/aug/test/masks/30_0.png'

In [43]:
print (pred_mask[0], true_mask[0])

Colab Notebooks/prediction/aug/118_0.png /content/drive/MyDrive/Dataset/aug/test/masks/118_0.png


In [44]:
score = []
for pred_y, true_y in tqdm(zip(pred_mask, true_mask), total=len(pred_mask)):
  name = pred_y.split ("/")[-1]
  pred_y = cv2. imread (pred_y, cv2.IMREAD_GRAYSCALE)
  pred_y = pred_y/255
  pred_y = pred_y > 0.5
  pred_y = pred_y.astype(np.int32)
  pred_y = pred_y.flatten()
  true_y = cv2.imread (true_y, cv2.IMREAD_GRAYSCALE)
  true_y = true_y/255.0
  true_y = true_y > 0.5
  true_y = true_y.astype(np.int32)
  true_y = true_y.flatten()

  acc_value = accuracy_score (pred_y, true_y)
  f1_value = f1_score(pred_y, true_y, labels=[0, 1], average="binary")
  jac_value = jaccard_score(pred_y, true_y, labels=[0, 1], average="binary")
  recall_value = recall_score(pred_y, true_y, labels=[0, 1], average="binary")
  precision_value = precision_score(pred_y, true_y, labels=[0, 1], average="binary")
  score.append ([name, acc_value, f1_value, jac_value, recall_value, precision_value])

100%|██████████| 19/19 [00:22<00:00,  1.19s/it]


In [45]:
score = [s[1:] for s in score]

In [46]:
mean_score = np.mean(score, axis=0)

In [47]:
mean_score

array([0.87141338, 0.63659348, 0.51450034, 0.80402727, 0.6948478 ])

In [48]:
print(f"Accuracy: {mean_score[0]:0.5f}")
print(f"F1: {mean_score[1]:0.5f}")
print(f"Jaccard: {mean_score[2]:0.5f}")
print(f"Recall: {mean_score[3]:0.5f}")
print(f"Precision: {mean_score[4]:0.5f}")

Accuracy: 0.87141
F1: 0.63659
Jaccard: 0.51450
Recall: 0.80403
Precision: 0.69485
